In [3]:
SEASON = '2013-14'

In [4]:
import pandas as pd
import myconstants as c
from scipy.stats import zscore

NUM_GAMES = c.NUM_GAMES[SEASON]
MIN_GAMES = NUM_GAMES * 0.60

p_df = pd.read_csv(f'../data/years/{SEASON}/clean/{SEASON}_Player_PerGame.csv')
t_df = pd.read_csv(f'../data/years/{SEASON}/clean/{SEASON}_Team_Standings.csv')

# join player per game and advanced stats
p_advanced_df = pd.read_csv(f'../data/years/{SEASON}/clean/{SEASON}_Player_Advanced.csv')
p_advanced_df = p_advanced_df.drop(columns=['Player', 'Tms', 'Pos', 'Age', 'G', 'MP'])
p_df = p_df.merge(p_advanced_df, on='PlayerID')

# populate team df with team codes
t_df['TeamCode'] = t_df['Team'].apply(lambda x: c.TEAM_CODE[x])

# average team wins per player
def average_wins(tms):
    tms_list = tms.split(',')
    tms_wins = []
    for t in tms_list:
        tms_wins.append(t_df.loc[t_df['TeamCode'] == t]['OverallW'].values[0])
    return sum(tms_wins) / len(tms_wins)
p_df['TmsWins'] = p_df['Tms'].apply(average_wins)

# players who haven't played enough games don't qualify
p_df = p_df.loc[p_df['G'] >= MIN_GAMES]

# ranking algorithm
p_df['PlayerScore'] = 1.5*zscore(p_df['PTS'] + p_df['TRB'] + p_df['AST'] + p_df['STL'] + p_df['BLK']) + zscore(p_df['WS']) + zscore(p_df['VORP']) + zscore(p_df['BPM']) + zscore(p_df['TmsWins'])
p_df = p_df.sort_values('PlayerScore', ascending=False)

# Move [PlayerScore] to the right of [Pos], drop PlayerID, index rows starting from 1, stylistic choice
col = p_df.pop('PlayerScore')
p_df.insert(p_df.columns.get_loc('Pos') + 1, col.name, col)
p_df = p_df.drop('PlayerID', axis=1)
p_df = p_df.reset_index(drop=True)
p_df.index = p_df.index + 1

p_df.head(50)

,Player,Tms,Pos,PlayerScore,Age,G,GS,MP,FG,FGA,...,USG%,OWS,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP,TmsWins
1,Kevin Durant,OKC,SF,21.479047,25,81,81,38.5,10.5,20.8,...,33.0,14.8,4.4,19.2,0.295,8.8,1.4,10.2,9.6,59.0
2,LeBron James,MIA,PF,17.445055,29,77,77,37.7,10.0,17.6,...,31.0,12.3,3.7,15.9,0.264,7.8,1.1,8.8,7.9,54.0
3,Kevin Love,MIN,PF,16.041208,25,77,77,36.3,8.4,18.5,...,28.8,10.6,3.7,14.3,0.245,8.3,0.6,8.9,7.7,40.0
4,Stephen Curry,GSW,PG,14.370337,25,78,78,36.5,8.4,17.7,...,28.3,9.3,4.0,13.4,0.225,6.3,1.1,7.4,6.7,51.0
5,Chris Paul,LAC,PG,14.002320,28,62,62,35.0,6.5,14.0,...,23.7,9.0,3.2,12.2,0.270,6.0,2.6,8.6,5.8,57.0
6,James Harden,HOU,SG,12.696653,24,73,73,38.0,7.5,16.5,...,27.8,10.1,2.7,12.8,0.221,5.4,0.2,5.6,5.3,54.0
7,Blake Griffin,LAC,PF,12.100254,24,80,80,35.8,9.0,17.0,...,29.0,8.1,4.1,12.2,0.205,3.8,0.7,4.5,4.7,57.0
8,Paul George,IND,SF,11.006216,23,80,80,36.2,7.2,17.0,...,28.3,4.4,6.4,10.8,0.178,2.7,2.1,4.9,5.0,56.0
9,Carmelo Anthony,NYK,PF,10.770393,29,77,77,38.7,9.6,21.3,...,32.4,8.1,2.5,10.7,0.172,5.8,-0.8,5.0,5.3,37.0
10,Joakim Noah,CHI,C,10.385959,28,80,80,35.3,4.8,10.0,...,18.7,4.5,6.6,11.2,0.190,1.7,3.6,5.3,5.2,48.0
